<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-nc-sa/4.0/80x15.png" /></a>&nbsp;| Luca Mossina and [Emmanuel Rachelson](https://personnel.isae-supaero.fr/emmanuel-rachelson?lang=en) | <a href="https://supaerodatascience.github.io/machine-learning/">https://supaerodatascience.github.io/machine-learning/</a>

<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">An application of SVMs in Multi-Label Classification (MLC)</div>

We'll see an application which is both harder and less common than binary classification, that of **multi-label classification** (MLC).  
Given a list of possible labels, the problem consists in finding one **or more** labels associated to a data point.  
For instance, imagine extracting the key topics from a newspaper article, or classifing the elements composing an image. Possibly many labels can be associated to each item.

Given a set of labels $\mathcal{L} = \{l_1, l_2, ..., l_k\} \in \{0,1\}^k$, we want to map elements of a feature space $\mathcal{X}$ to a subset of $\mathcal{L}$:  

$$h : \mathcal{X} \longrightarrow \mathcal{P}(\mathcal{L})$$

The two typical approaches for such problems are known as **Binary Relevance** (BR) and **Label Powerset** (LP).  

 - BR: each label in $\mathcal{L}$ is a binary classification problem, $h_{i} : \mathcal{X} \longrightarrow l_{i}, l_{i} \in \{0,1\}, i = 1, ..., |\mathcal{L}|$.  
 This method ignores any correlation between labels (supposes them independent).

 - LP: transforms a problem of MLC into one of multiclass classification, mapping elements $x \in \mathcal{X}$ directly to $s \in \mathcal{P}(\mathcal{L})$.  
 This method becomes rapidly inapplicable as the number of elemnts in $\mathcal{P}(\mathcal{L})$ grows exponentially with the number of labels.
 
If you are curious on the topic of MLC, you are encouraged to read these references:  
J. Read, P. Reutemann, B. Pfahringer, and Geoff Holmes. **MEKA: A multi-label/multi-target extension to Weka**. Journal of Machine Learning Research, 17(21):1-5, 2016.  
G. Tsoumakas and I. Katakis. **Multi-label classification: An overview**. International Journal on Data Warehousing and Mining, 3(3):1-13, 2007.  
G. Tsoumakas, I. Katakis, and I. Vlahavas. **Mining multi-label data**. Data mining and knowledge discovery handbook, pages 667-685. Springer, 2010.
 
Many other variations exist, but for today we'll focus on BR, the most straightforward to implement. What we will start implementing below is a good start if you want to explore what is done in:  
J. Read, B. Pfahringer, G. Holmes, and E. Frank. **Classifier chains for multi-label classification**. Joint European Conference on Machine Learning and Knowledge Discovery in Databases, pages 254-269, 2009.

The equivalent approach for LP is found in:  
G. Tsoumakas, I. Katakis, and I. Vlahavas. **Random k-labelsets for multi-label classification**. IEEE Transactions on Knowledge and Data Engineering, 23(7):1079-1089, 2011.

For this exercise, we will use a biology dataset from [Elisseeff and Weston 2001]: this dataset contains micro-array expressions and phylogenetic profiles for 2417 yeast genes. Each gene is annotated with a subset of 14 functional categories (e.g. metabolism, energy, etc.) of the top level of the functional catalogue.

<div class="alert alert-warning">

**Exercice**<br>
<ul>

<li> find a suitable package to load the file at `yeast.arff`.  <br>
    Hint: <a href=https://docs.scipy.org/doc/scipy/reference/io.html>scipy.io</a> and _read the doc_.<br>
<li> Store the data in a pandas dataframe.<br>
    Hint: columns of classes will be encoded as 'utf-8', we need integers, look for 'str.decode('utf-8')'
<li> check dataset: you should have 2417 samples $\times$ 117 columns (103 features + 14 labels)
</ul>
</div>

In [3]:
# %load solutions/code1.py
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).

# Read arff data
import pandas as pd
import scipy
from scipy.io import arff

# Load yeast.arff via dedicated scipy.io function
raw_data, metadata = arff.loadarff('../data/yeast/yeast.arff')
print("nrows:", len(raw_data))    # 2417
print("ncols:", len(raw_data[0])) #  117

# Data to pandas, converting unicode columns to integers
df = pd.DataFrame(raw_data)
# print(df.shape)           # -> (2417, 117)
# print(df.head(5))         # for free, we get column names
# print(type(df.iloc[0,0])) # -> <class 'bytes'> ## we want to have plain {0,1} integers

classes_list = [name for name in df.columns if "Class" in name]
# print(classes_list)  # -> ['Class1', 'Class2', ... , 'Class14']

for col in df[classes_list]:
    df[col] = (df[col].str.decode('utf-8').astype(int))

print(type(df.iloc[0,0]))  # -> int: as expected
print(type(df.iloc[0,15])) # -> float: as expected
print("dataframe dimensions:", df.shape)    # -> (2417, 117)


nrows: 2417
ncols: 117
<class 'numpy.int32'>
<class 'numpy.float64'>
dataframe dimensions: (2417, 117)


<div class="alert alert-warning">

**Exercice**<br>
<ul>
<li> Manually, fit a SVM classifier for each label in the dataset
<li> Apply a cross-validation of 60 ∕ 40: 60% of datapoints to train the model, 40% to test it  <br>
   Remember: it is good practice to <b>randomly shuffle</b> the data, in case the data are ordered w.r.t. some data-dependent criterion.
<li> Report some performance measure
</ul>
</div>

In [4]:
# %load solutions/code2.py
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).

# === Hints:
CVRATIO = 0.4

# === Solution:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# Get a list of all features: "Att1" ... "Att103"
features_list = [name for name in df.columns if "Att" in name]

# Shuffle rows
df = df.sample(frac=1)

# Shown classifier n.1. To you to put it in a nice for loop and store the results
X_train, X_test, y_train, y_test = train_test_split(
    df[features_list],
    df[classes_list],
    test_size=0.4,
    random_state=0 # <- random seed, could be useful to replicate previous results
)

clf = SVC(gamma='auto')
clf.fit(X_train, y_train["Class1"])

y_pred_01 = clf.predict(X_test)



**Congratulations**, you reached the end of the practice session! 